# Importe
Neben Paketen die standardmäßig bei Anaconda enthalten sind, werden auch externe Pakete wie z.B. gensim.word2vec geladen. Weiterhin werden externe Dateien wie z.B. ein Word2Vec und ein Spacy-Modell geladen.

In [1]:
import datetime
import pandas as pd
import numpy as np
import spacy
import string
import Levenshtein
import gensim
import networkx as nx
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from itertools import permutations
import singularize

stop = stopwords.words('german')
stop.remove('von')
stop.remove('vom')
nlp = spacy.load('de_core_news_md')
model = gensim.models.KeyedVectors.load_word2vec_format("german.model", binary = True)
vocabulary = set(model.wv.vocab.keys())

D:\Programme\Anaconda\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


# Relationen-Dictionary laden
Um Synonyme und alternative Umschreibungen für Relationen zu erfassen, wurde ein Wörterbuch für typische Relationssynonyme angelegt. Oftmals werden für ein und dieselbe Relation Synonyme (z.B. Lehrstuhlinhaber -> Lehrstuhlleitung) verwendet. Durch Wörterbuch basiertes Austauschen sollen die Relationen vereinheitlicht werden. Das hierfür benötigte Dictionary wird nun erstellt.

In [2]:
relation_dict = {
'vorlesungsnummer' : 'Vorlesungsnummer',
'veranstaltungsart' : 'Veranstaltungsart',
'dozent' : 'Dozent',
'veranstaltung von' : 'Veranstaltung von',
'head of research group' : 'Lehrstuhlinhaber',
'telefon' : 'Telefon',
'phone' : 'Telefon',
'room' : 'Raum',
'raum' : 'Raum',
'gebäude' : 'Raum',
'sprechstunde' : 'Sprechstunde',
'consultation' : 'Sprechstunde',
'consultation-hour' : 'Sprechstunde',
'consultation-hours' : 'Sprechstunde',
'hours' : 'Sprechstunde',
'e-mail' : 'E-Mail',
'email' : 'E-Mail',
'zimmer' : 'Raum',
'juniorprofessor' : 'Lehrstuhlinhaber',
'juniorprofessorin' : 'Lehrstuhlinhaber',
'juniorprofessur' : 'Lehrstuhlinhaber',
'professur' : 'Lehrstuhlinhaber',
'lehrbeauftragterin' : 'Wissenschaftlicher Mitarbeiter',
'Building' : 'raum',
'office' : 'Sprechstunde',
'secretary' : 'Sekretariat',
'sekretärin' : 'Sekretariat',
'technician' : 'Techniker', 
'graduate student' : 'Studentischer Mitarbeiter',
'undergraduate student' : 'Studentischer Mitarbeiter',
'phd student' : 'Doktorand',
'teacher' : 'Wissenschaftlicher Mitarbeiter',
'professor' : 'Lehrstuhlinhaber',
'consulation' : 'Sprechstunde',
'group leader' : 'Lehrstuhlinhaber',
'academic councillor' : 'Akademischer Rat',
'academic counselor' : 'Akademischer Rat',
'secretariat' : 'Sekretariat',
'technical assistant' : 'Techniker',
'phd' : 'Doktorand',
'doktorandin' : 'Doktorand',
'doktorant' : 'Doktorand',
'doktorantin' : 'Doktorand',
'sprechzeit' : 'Sprechstunde',
'wissenschaftlicher mitarbeiter' : 'Wissenschaftlicher Mitarbeiter',
'wissenschaftliche mitarbeiterin' : 'Wissenschaftlicher Mitarbeiter',
'wiss. mitarbeiter' : 'Wissenschaftlicher Mitarbeiter',
'wiss. mitarbeiterin' : 'Wissenschaftlicher Mitarbeiter',
'studentische hilfskraft' : 'Studentischer Mitarbeiter',
'lehrstuhlinhaberin' : 'Lehrstuhlinhaber',
'sekretariat' : 'Sekretariat',
'tel' : 'Telefon',
'mail' : 'E-Mail',
'fax' : 'Fax',
'fox' : 'Fax',
'research assistant' : 'Wissenschaftlicher Mitarbeiter',
'master studentin' : 'Studentischer Mitarbeiter',
'master student' : 'Studentischer Mitarbeiter',
'akad. rat a.z' : 'Akademischer Rat',
'akad. rat. a.z.' : 'Akademischer Rat',
'chairholder' : 'Lehrstuhlinhaber',
'acad. director' : 'Akademischer Rat',
'staff scientist' : 'Wissenschaftlicher Mitarbeiter',
'ph.d. student' : 'Doktorand',
'ph d student' : 'Doktorand',
'ph d. student' : 'Doktorand',
'bachelor student' : 'Studentischer Mitarbeiter',
'bachelor studentin' : 'Studentischer Mitarbeiter',
'lehrkraft' : 'Wissenschaftlicher Mitarbeiter',
'physical engineer' : 'Wissenschaftlicher Mitarbeiter',
'vertretungsprofessur' : 'Vertretungsprofessur',
'doktorand' : 'Doktorand',
'sprechzeit' : 'Sprechstunde',
'sprechzeiten' : 'Sprechstunde',
'Sprechstunden' : 'Sprechstunde',
'akademische rätin' : 'Akademischer Rat',
'lehrstuhlinhaber' : 'Lehrstuhlinhaber',
'lehrstuhlinhaberin' : 'Lehrstuhlinhaber',
'wissenschaftliche hilfskraft' : 'Wissenschaftlicher Mitarbeiter',
'inhaberin' : 'Lehrstuhlinhaber',
'inhaber' : 'Lehrstuhlinhaber',
'professorin' : 'Lehrstuhlinhaber',
'adresse' : 'Raum',
'technische assistentin' : 'Wissenschaftlicher Mitarbeiter',
'technischer assistent' : 'Wissenschaftlicher Mitarbeiter',
'student assistant' : 'Studentischer Mitarbeiter',
'teamassistenz' : 'Wissenschaftlicher Mitarbeiter',
'academic council' : 'Akademischer Rat',
'lehrbeauftragte' : 'Lehrstuhlinhaber',
'lehrbeauftragter' : 'Lehrstuhlinhaber',
'projektmitarbeiter' : 'Wissenschaftlicher Mitarbeiter',
'projektmitarbeiterin' : 'Wissenschaftlicher Mitarbeiter',
'büro' : 'Raum',
'building' : 'Raum',
'akademischer rat' : 'Akademischer Rat',
'akademische rätin' : 'Akademischer Rat',
'akademischer oberrat' : 'Akademischer Rat',
'akademische oberrätin' : 'Akademischer Rat',
'akademischer direktor' : 'Akademischer Direktor', 
'akademic director' : 'Akademischer Direktor', 
'alumni' : 'Alumni',
'bürozeiten' : 'Sprechstunde',
'laboratory assistant' : 'Wissenschaftlicher Mitarbeiter',
'post-doc' : 'Wissenschaftlicher Mitarbeiter',
'post doc' : 'Wissenschaftlicher Mitarbeiter',
'masterand' : 'Studentischer Mitarbeiter',
'masterandin' : 'Studentischer Mitarbeiter',
'bachelorand' : 'Studentischer Mitarbeiter',
'bachelorandin' : 'Studentischer Mitarbeiter',
'assistent' : 'Wissenschaftlicher Mitarbeiter',
'assistentin' : 'Wissenschaftlicher Mitarbeiter',
'student' : 'Studentischer Mitarbeiter',
'sprechstunde' : 'Sprechstunde',
'öffnungszeit' : 'Sprechstunde',
'öffnungszeiten' : 'Sprechstunde',
'mitarbeiterin' : 'Wissenschaftlicher Mitarbeiter',
'mitarbeiter' : 'Wissenschaftlicher Mitarbeiter',
'angestellter' : 'Wissenschaftlicher Mitarbeiter',
'habilitandin' : 'Habilitand',
'habilitand' : 'Habilitand',
'address' : 'Raum', 
'sprechstunden' : 'Sprechstunde',
'honorarprofessor' : 'Honorarprofessor',
'honorarprofessorin' : 'Honorarprofessor',
'lehrstuhlvertretung' : 'Lehrstuhlvertretung',
'mitarbeiter' : 'Wissenschaftlicher Mitarbeiter',
'mitarbeiterin' : 'Wissenschaftlicher Mitarbeiter',
'leiter' : 'Leiter',
'leiterin' : 'Leiter',
'austausch-doktorand' : 'Doktorand',
'bachelor-student' : 'Studentischer Mitarbeiter',
'bachelor-studentin' : 'Studentischer Mitarbeiter',
'master-student' : 'Studentischer Mitarbeiter',
'master-studentin' : 'Studentischer Mitarbeiter',
'postdoc' : 'Wissenschaftlicher Mitarbeiter',
'head of research' : 'Lehrstuhlinhaber',
'homepage' : 'Webseite',
'website' : 'Webseite',
'webseite' : 'Webseite',
'studentische hilfskräfte' : 'Studentischer Mitarbeiter',
'research scientist' : 'Wissenschaftlicher Mitarbeiter',
'lecturer' : 'Wissenschaftlicher Mitarbeiter',
'telefax' : 'Fax',
'assistant' : 'Wissenschaftlicher Mitarbeiter',
'privatdozent' : 'Privatdozent',
'privatdozentin' : 'Privatdozent',
'arbeitsgruppenleiter' : 'Arbeitsgruppenleiter',
'leitung' : 'Leitung',
'head' : 'Leitung',
'head of chair' : 'Lehrstuhlinhaber',
'wissenschaftliche' : 'Wissenschaftlicher Mitarbeiter',
'wissenschaftlicher' : 'Wissenschaftlicher Mitarbeiter',
'telefonsprechstunde' : 'Sprechstunde',
'studierendensekretariat' : 'Sekretariat',
'promovendin' : 'Doktorand',
'promovierende' : 'Doktorand',
'promovend' : 'Doktorand',
'promovierender' : 'Doktorand',
'fachleitung' : 'Leitung',
'techniker' : 'Techniker',
'technikerin' : 'Techniker',
'fachleiter' : 'Leitung',
'fachleiterin' : 'Leitung',
'lehrstuhlsekretärin' : 'Sekretariat',
'lektor' : 'Wissenschaftlicher Mitarbeiter',
'w1-juniorprofessorin' : 'Lehrstuhlinhaber',
'associate' : 'Wissenschaftlicher Mitarbeiter',
'studentische' : 'Studentischer Mitarbeiter',
'studiengangsmoderator' : 'Studiengangsmoderator',
'studiengangsmoderatorin' : 'Studiengangsmoderator',
'dekan' : 'Dekan',
'verwaltungsangestellte' : 'Sekretariat',
'technik' : 'Techniker',
'reinraumtechniker' : 'Techniker',
'emeritus' : 'Emeritus',
'principal investigator' : 'Wissenschaftlicher Mitarbeiter',
'studiengangskoordinatorin' : 'Studiengangskoordinator',
'studiengangskoordinator' : 'Studiengangskoordinator',
'oder' : 'E-Mail',
'gründungsdekan' : 'Dekan',
'geschäftsführer' : 'Geschäftsführer',
'appointments' : 'Sprechstunde',
'appointment' : 'Sprechstunde',
'scholar' : 'Wissenschaftlicher Mitarbeiter',
'telefonnummer' : 'Telefon',
'e-mail adresse' : 'E-Mail',
'büro' : 'Raum',
'veranstaltungen' : 'Veranstaltung von',
'veranstaltung' : 'Veranstaltung von',
'faxnummer' : 'Fax',
'fächer' : 'Veranstaltung von',
'klausur' : 'Klausur',
'prüfung' : 'Klausur',
'abschlussprüfung' : 'Klausur',
'test' : 'Klausur',
'exam' : 'Klausur',
'examen' : 'Klausur',
'mensa' : 'Mensa',
'essen' : 'Mensa',
'kantine' : 'Mensa',
'cantine' : 'Mensa',
'cafeteria' : 'Mensa',
'mittagessen': 'Mensa',
'cafe' : 'Mensa',
'pause' : 'Mensa',
'frühstuck' : 'Mensa',
'abendessen' : 'Mensa',
'hunger' : 'Mensa',
'mittag' : 'Mensa',
'durst' : 'Mensa',
'elearning' : 'e-Learning',
'e-learning' : 'e-Learning',
'link' : 'e-Learning',
'links' : 'e-Learning'
}


def enhance_relation_dict(relation_dict):
    """Erstellt zusätzliche Einträge für Alternativschreibweisen von relationen.
    Es wird für jeden bestehenden Eintrag ein Eintrag kleingeschrieben und mit z.B.
    'ae' statt 'ä' gespeichert."""
    dict_values = relation_dict.copy().values()
    dict_keys = relation_dict.copy().keys()
    for val in dict_values:
        relation_dict[val.lower()] = val
    for key in dict_keys:
        temp = key
        if ('ä' in temp) or ('Ä' in temp):
            temp = temp.replace('ä', 'ae')
            temp = temp.replace('Ä', 'Ae')
        if ('ö' in temp) or ('Ö' in temp):
            temp = temp.replace('ö', 'oe')
            temp = temp.replace('Ö', 'Oe')
        if ('ü' in temp) or ('Ü' in temp):
            temp = temp.replace('ü', 'ue')
            temp = temp.replace('Ü', 'Ue')
        relation_dict[temp] = relation_dict[key]
    return relation_dict

relation_dict = enhance_relation_dict(relation_dict)

# Knowledge Graph laden
Der zuvor gescrapede Knowledge Graph wird nun importiert und angepasst (z.B. Übung & Vorlesung auf zwei Knoten entzerren usw.)

In [3]:
# KG laden
kg = pd.read_csv('knowledge_graph_relations_cleaned.csv', sep = '|')

# Knoten entfernen, die falsch sind
kg = kg[~(kg['source'].isin(relation_dict.values()) | kg['target'].isin(relation_dict.values()))]
kg = kg[~(kg['source'] == 'and / Wissenschaftlicher Mitarbeiter')]
kg = kg[(kg['source'].apply(len) > 2)]
kg = kg[(kg['target'].apply(len) > 2)]

# Vorlesung & Übung auf zwei Knoten entzerren
temp = kg[kg['target'] == 'Vorlesung & Übung'].copy()
temp.loc[:,'target'] = 'Vorlesung'
vorlesung_only = temp.copy()
temp = kg[kg['target'] == 'Vorlesung & Übung'].copy()
temp.loc[:,'target'] = 'Übung'
übung_only = temp.copy()
duplicated_vü = vorlesung_only.append(übung_only)
kg = kg[~(kg['target'] == 'Vorlesung & Übung')]
kg = kg.append(duplicated_vü).drop_duplicates()

# KG Einträge spiegeln und duplizieren -> Aus dem gerichteten Graphen wird ein ungerichteter Graph
kg_reversed = kg[['target', 'relation', 'source']]
kg_reversed.columns = ['source', 'relation', 'target']
kg = kg.append(kg_reversed).drop_duplicates()
kg = kg.reset_index(drop=True)

# Variablen für späteren Zugriff
nodes = set(kg.source.to_list() + kg.target.to_list())
nodes_unchanged = kg.source.to_list()

# CountVectorizer berechnen
Um Fragen und Antworten auch außerhalb des Wörterbuchs vergleichbar zu machen, wird nun ein CountVectorizer auf den Knoten des Knowledge Graphs berechnet

In [4]:
def clean_text(text):
    """Bereinigt einen Text indem es ihn zu Kleinbuchstaben
    transformiert, Satzzeichen und Zahlen entfernt, generelle Stopwords entfernt
    und Deklinationen entfernt."""
    text = str(text)
    spacy_doc = nlp(text)
    text = ''
    for token in spacy_doc:
        text = text + token.lemma_ + ' '
    text = text.lower().strip()
    text = ''.join([x for x in text if x not in list(string.punctuation)])
    text = ''.join([x for x in text if not x.isdigit()])
    text = ' '.join([x for x in text.split() if x not in list(stop)])
    spacy_doc = nlp(text)
    text = ''
    for token in spacy_doc:
        text = text + token.lemma_ + ' '
    return text.strip()

def create_cleaned_corpus(kg):
    """Erstellt ein bereinigtes Corpus für die Knoten eines KG."""
    corpus = kg['source'].to_list()
    corpus_clean = []
    for doc in tqdm(corpus):
        corpus_clean.append(clean_text(doc))
    return corpus_clean


corpus_source_clean = create_cleaned_corpus(kg)

vectorizer = CountVectorizer()
vectorizer.fit(corpus_source_clean)
vectorizer_vocab = set(vectorizer.vocabulary_.keys())

100%|████████████████████████████████████████████████████████████████████████████| 30357/30357 [10:36<00:00, 47.69it/s]


# Count-Vektoren vorberechnen
Um Berechnungszeit vorweg zu nehmen, sollen nun für alle Knoten die Count-Vektoren berechnet und in einem Dictionary abgespeichert werden. Dies hat eine erhebliche Performanceverbesserung zur Folge.

In [5]:
count_vectors = {}
for i in range(len(corpus_source_clean)):
    count_vectors[i] = vectorizer.transform([corpus_source_clean[i]])

# Antworten auf eine Frage im Knowledge Graph finden. 
Im Folgenden findet sich das System, welches Antworten auf Fragen im Knowledge Graph sucht. Das Vorgehen ist dabei wie folgt:
1. Relation ermitteln
2. Entität ermitteln
3. gesuchte Knoten lokalisieren

In [6]:
def get_answer(question):
    """Ermittelt für eine Frage question die beste Antwort result."""
    relations = get_relation(question)
    if len(relations) > 0:
        minimum_entities = 1
    else:
        minimum_entities = 2
    entities = get_entities(question, minimum_entities)
    noun_chunk_result = get_result(entities, relations, question)
    return noun_chunk_result

## Relation ermitteln
Vorgehen:
1. Für alle Wortpermutationen einer Frage wird überprüft ob diese bereits bekannte Relationen sind
2. Falls nicht, so wird nach ähnlichen bekannten Relationen mit Levenshtein-Distanz kleiner 2 gesucht.
3. Falls immer noch keine Relationen gefunden wurden, so werden die 50 ähnlichsten Wörter laut Word2Vec eingeholt und überprüft ob diese bekannte Relationen sind

In [7]:
def get_relation(question):
    """Ermittelt die gesuchte relation einer Frage."""
    question_cleaned = clean_question_for_relation(question)
    relations = check_existing_relations(question_cleaned)
    if len(relations) == 0:
        relations = get_levenshtein_relation(question_cleaned)
    if len(relations) == 0:
        relations = get_word2vec_relation(question)
    return relations

def clean_question_for_relation(question):
    """Bereinigt eine Frage so, dass die zugrundeliegende
    Relation gefunden werden kann."""
    question = ''.join([x for x in question if x not in string.punctuation])
    question = [x.strip().lower() for x in question.split()]
    question = [x for x in question if x not in stop]
    return question

def check_existing_relations(candidates):
    """Überprüft für eine Liste von Kandidaten, ob sie eine
    bereits bestehende Relation ist."""
    relations = []
    candidates = get_permutations(candidates, 2, 0)
    for word in candidates:
        try:
            relations.append(relation_dict[word])
        except:
            continue
    return list(set(relations))

def get_levenshtein_relation(words):
    """Ermittelt ähnliche Relationen für eine Liste von Wörtern. Eine 
    ähnliche Relation liegt vor bei Levenshtein-Distanz kleiner 2."""
    similar_relations = []
    relations = list(set(relation_dict.values()))
    words = get_permutations(words, 2, 0)
    for word in words:
        for relation in relations:
            length_difference = abs(len(word) - len(relation))
            if length_difference < 3:
                levenshtein_distance_cleaned = Levenshtein.distance(word, relation.lower())
                levenshtein_distance_singular = Levenshtein.distance(singularize.singularize(word), relation.lower())
                if (levenshtein_distance_cleaned < 2) or (levenshtein_distance_singular < 2):
                    similar_relations.append(relation)
    return similar_relations

def get_permutations(word_list, high, low):
    """Generiert alle Permutationen für eine Liste von Wörtern."""
    perms = []
    for i in range(high, low, -1):
        perm = permutations(word_list, i)
        for j in perm:
            perm_joined = ' '.join(j)
            perms.append(perm_joined)
    return perms

def get_word2vec_relation(question):
    """Ermittelt die am besten passende Relation, indem
    ähnliche Wörter ermittelt werden und überprüft wird ob
    jene bekannt sind."""
    question = ''.join([x for x in question if x not in string.punctuation])
    question = [x.strip() for x in question.split() if x not in stop]
    best_fit = []
    for word in question:
        if word in vocabulary:
            most_similar = model.most_similar(word, topn = 50)
            most_similar = [x[0] for x in most_similar]
            for similar_word in most_similar:
                existing_relation = check_existing_relations([similar_word.lower()])
                best_fit.extend(existing_relation)
    if len(best_fit) > 1:
        return [Counter(best_fit).most_common(1)[0][0]]
    else:
        return best_fit

## Entities finden
Vorgehen:
1. Noun_chunks ermitteln und überprüfen ob diese eine Entität darstellen.
2. Falls nicht, so wird für alle Fragepermutationen überprüft ob ein bestehender Knoten enthalten ist
3. Falls nicht, so wird für alle Fragepermutationen überprüft ob ein bestehender Knoten mit Levenshtein-Distanz kleiner 2 bekannt ist
4. Falls nicht, so wird mit Cosinus-Similarity der Count-Vektoren der Knoten und der Frage der beste Knoten ermittelt

In [8]:
def get_entities(question, minimum_entities):
    """Extrahiert alle Entitäten einer Frage."""
    question = translate_weekdays(question)
    noun_chunks = get_noun_chunks(question)
    entities = filter_if_noun_chunk_is_entity(noun_chunks)
    if len(entities) >= minimum_entities:
        return entities
    else:
        return get_question_entities(question, minimum_entities)

def translate_weekdays(text):
    """Übersetzt z.B. morgen zu einem Wochentag"""
    weekday_dict = {6:'Sonntag', 0:'Montag', 1:'Dienstag',
                    2:'Mittwoch', 3:'Donnerstag', 4:'Freitag',
                    5:'Samstag', -2:'Samstag', -1:'Sonntag',
                    7:'Montag', 8:'Dienstag'}
    if 'heute' in text.lower():
        weekday =  weekday_dict[datetime.datetime.today().weekday()]
        text = text.replace('heute', weekday)
        text = text.replace('Heute', weekday)
    if 'gestern' in text.lower():
        weekday = weekday_dict[datetime.datetime.today().weekday() - 1]
        text = text.replace('gestern', weekday)
        text = text.replace('Gestern', weekday)
    if 'vorgestern' in text.lower():
        weekday = weekday_dict[datetime.datetime.today().weekday() - 2]
        text = text.replace('vorgestern', weekday)
        text = text.replace('Vorgestern', weekday)
    if 'morgen' in text.lower():
        weekday = weekday_dict[datetime.datetime.today().weekday() + 1]
        text = text.replace('morgen', weekday)
        text = text.replace('Morgen', weekday)
    if 'übermorgen' in text.lower():
        weekday = weekday_dict[datetime.datetime.today().weekday() + 2]
        text = text.replace('übermorgen', weekday)
        text = text.replace('Übermorgen', weekday)
    return text

def get_noun_chunks(question):
    """Extrahiert die Noun Chunks einer Frage als potenzielle
    Entitäten."""
    doc = nlp(question)
    entities = [str(chunk) for chunk in doc.noun_chunks]
    single_ents = []
    for ents in entities:
        single_ents.extend(ents.split(' '))
    return [x for x in single_ents if x.lower() not in stop]

def filter_if_noun_chunk_is_entity(candidates):
    """Überprüft für eine Liste von Kandiaten, ob sie eine
    bekannte Entität ist."""
    candidates = get_permutations(candidates, len(candidates), 0)
    known_entities = []
    for candidate in candidates:
        if candidate in nodes:
            known_entities.append(candidate)
    return known_entities

def get_question_entities(question, minimum_entities):
    """Ermittelt die Entitäten einer Frage, wenn die Suche über
    noun_chunks kein zufriedenstellendes Ergebnis geliefert hat."""
    entities = check_existing_entities(question)
    if len(entities) >= minimum_entities:
        return entities
    entities = levenshtein_compare_existing_entities(question)
    if len(entities) >= minimum_entities:
        return entities
    entities = get_count_vector_entities(question, entities, minimum_entities)
    return entities
        
def check_existing_entities(question):
    """Überprüft alle bestehenden Knoten auf Übereinstimmungen."""
    entities = []
    question = question.strip(string.punctuation)
    question_permutations = get_permutations(question.split(), 5,0)
    for perm in tqdm(question_permutations):
        if perm in nodes:
            entities.append(perm)
        else:
            singularized_perm = ' '.join([singularize.singularize(x) for x in perm.split()])
            if (singularized_perm in nodes):
                entities.append(singularized_perm)
            elif (singularized_perm.capitalize() in nodes):
                entities.append(singularized_perm.capitalize())
    return entities

def levenshtein_compare_existing_entities(question):
    """Vergleicht die Wörter einer Frage anhand der
    Levenshtein-Distanz mit bereits bestehenden Entitäten."""
    entities = []
    question = question.strip(string.punctuation)
    question_permutations = get_permutations(question.split(), 3,0)
    for perm in question_permutations:
        for node in nodes:
            length_difference = abs(len(perm) - len(node))
            if length_difference < 2:
                levenshtein_distance = Levenshtein.distance(perm, node)
                if levenshtein_distance < 2:
                    entities.append(node)
    return entities

def get_count_vector_entities(question, known_entities, required_entities):
    """Ermittelt den ähnlichsten Knoten für eine Frage 
    anhand von Count-Vektoren."""
    required_entities = required_entities - len(known_entities)
    if len(known_entities) == 1:
        known_entities = known_entities[0]
    else:
        known_entities = False
    question = clean_text(question)
    question = [' '.join([x for x in question.split() if x in vectorizer_vocab]).strip()]
    arr = vectorizer.transform(question)
    max_similarity = -999
    second_best_similarity = -9999
    index = -999
    second_best_index = -9999
    for i in range(len(count_vectors)):
        if known_entities and (nodes_unchanged[i] == known_entities):
            continue
        similarity = cosine_similarity(arr, count_vectors[i])
        if similarity > max_similarity:
            if required_entities == 2:
                if similarity > max_similarity:
                    second_best_similarity = max_similarity
                    second_best_index = index
                    max_similarity = similarity
                    index = i
                elif (similarity < max_similarity) and (similarity > second_best_similarity):
                    second_best_similarity = similarity
                    second_best_index = i
                else:
                    continue
            else:
                max_similarity = similarity
                index = i
    if required_entities == 2:
        if (second_best_similarity < 0.5):
            return [nodes_unchanged[index]]
        else:
            return [nodes_unchanged[index], nodes_unchanged[second_best_index]]
    if known_entities:
        if (max_similarity < 0.5):
            return [known_entities]
        else:
            return [known_entities, nodes_unchanged[index]]
    else:
        return [nodes_unchanged[index]]

# Ergebnis ermitteln

Mithilfe der zuvor ermittelten Informationen wird nun die Antwort auf die Frage im Knowledge Graph gesucht. Hierbei wird in folgende Fälle unterschieden: 
- 1 Entität, 0 Relation
- 1 Entität, 1 Relation
- 1 Entität, mehrere Relationen
- 2 Entität, 0 Relation

Für alle Fälle wird bei einem Ausgangsknoten der Knoten mit der geringsten Distanz und den passendsten Relationen/Entitäten ermittelt.

In [9]:
def get_result(entities, relations, question):
    """Ermittelt die Antwort für die gegebenen
    entitites und relations."""
    print(entities, relations)
    if (len(entities) == 2):
        result = get_result_2_entities_0_relations(entities)
    elif (len(entities) == 1) and (len(relations) == 1):
        result = get_result_1_entity_1_relation(entities, relations)
    elif (len(entities) == 1) and (len(relations) == 0):
        result = get_result_1_entity_0_relations(entities)
    elif (len(entities) == 1) and (len(relations) > 1):
        result = get_result_1_entity_multiple_relations(entities, relations)
    return result

def get_result_2_entities_0_relations(entities):
    """Ermittelt das Ergebnis, im Falle, dass zwei Entitäten
    und keine Relation bekannt sind."""
    node_to_look_for = [entities[0]]
    count = 0
    while (count < 5):
        count += 1
        new_set = kg[kg['source'].isin(node_to_look_for)]
        node_to_look_for = new_set.target.to_list()
        if entities[1] in node_to_look_for:
            result = new_set[new_set['target'] == entities[1]]
            return drop_duplicates_result_set(result)
    return False

def get_result_1_entity_1_relation(entities, relations):
    """Ermittelt das Ergebnis, im Falle, dass eine Entität
    und eine Relation bekannt sind."""
    count = 0
    node_to_look_for = entities.copy()
    while (count < 5):
        count += 1
        new_set = kg[kg['source'].isin(node_to_look_for)]
        node_to_look_for = new_set.target.to_list()
        if relations[0] in new_set.relation.to_list():
            result = new_set[new_set['relation'] == relations[0]]
            return drop_duplicates_result_set(result)
    return False

def get_result_1_entity_0_relations(entities):
    """Ermittelt das Ergebnis, im Falle, dass eine Entität
    und keine Relation bekannt sind."""
    result = kg[kg['source'] == entities[0]]
    return drop_duplicates_result_set(result)

def get_result_1_entity_multiple_relations(entities, relations):
    """Ermittelt das Ergebnis, im Falle, dass eine Entität
    und mehrere Relationen bekannt sind."""
    count = 0
    node_to_look_for = entities.copy()
    while (count < 5):
        count += 1
        new_set = kg[kg['source'].isin(node_to_look_for)]
        node_to_look_for = new_set.target.to_list()
        for relation in relations:
            if relation in new_set.relation.to_list():
                new_relations = [x for x in relations if x != relation]
                new_entities = new_set.target.to_list()
                return get_result_1_entity_1_relation(new_entities, new_relations)
    return False

def drop_duplicates_result_set(result):
    """Entfernt gespiegelte Dupliakte aus dem Antwort
    DataFrame."""
    to_drop = []
    done = []
    for i in range(len(result)):
        if result.iloc[i].name in done:
            continue
        source = result.iloc[i, 0]
        relation = result.iloc[i, 1]
        target = result.iloc[i, 2]
        mirror = result[(result['source'] == target) & (result['relation'] == relation) & (result['target'] == source)]
        if len(mirror) > 0:
            to_drop.append(mirror.index[0])
            done.append(mirror.index[0])
            done.append(result.iloc[i].name)
    result = result.drop(to_drop)
    return result

## Beispielhafte Anfragen

In [10]:
%%time
get_answer("Was gibt es Montag in der Mensa?")

['Montag'] ['Mensa']
Wall time: 177 ms


,source,relation,target
14927,Montag,Mensa,Cremiges Kohlrabi-Paprika-Currygulasch an gedr...


In [11]:
%%time
get_answer('Welche Veranstaltungen gibt es vom Lehrstuhl für Marketing und Innovation?')

100%|██████████████████████████████████████████████████████████████████████████| 36100/36100 [00:08<00:00, 4448.41it/s]


['Lehrstuhl für Marketing & Innovation'] ['Veranstaltung von']
Wall time: 43.3 s


,source,relation,target
19848,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Abschlussarbeitenkolloquium Innovations- und D...
20092,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Doktoranden- und Habilitandenseminar Innovatio...
20409,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Forschungsprojekt Data Mining im Marketing mit R
20413,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Forschungsprojekt „Innovations- und Dialogmark...
20417,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Forschungsprojekt Innovations- und Dialogmarke...
20611,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Hauptseminar Innovations- und Dialogmarketing
21177,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Seminar Innovations- und Dialogmarketing
21420,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Übung Dialogmarketing
21703,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Vorlesung Dialogmarketing


In [12]:
%%time
get_answer('Was gibt es für Veranstaltungen von Daniel Baier?')

['Daniel Baier'] ['Veranstaltung von']
Wall time: 41 ms


,source,relation,target
4671,Abschlussarbeitenkolloquium Innovations- und D...,Veranstaltung von,Lehrstuhl für Marketing & Innovation
4915,Doktoranden- und Habilitandenseminar Innovatio...,Veranstaltung von,Lehrstuhl für Marketing & Innovation
5232,Forschungsprojekt Data Mining im Marketing mit R,Veranstaltung von,Lehrstuhl für Marketing & Innovation
5240,Forschungsprojekt Innovations- und Dialogmarke...,Veranstaltung von,Lehrstuhl für Marketing & Innovation
5345,Grundlagen Marketing- und Dienstleistungsmanag...,Veranstaltung von,Lehrstuhl für Marketing & Dienstleistungsmanag...
6526,Vorlesung Dialogmarketing,Veranstaltung von,Lehrstuhl für Marketing & Innovation
20413,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Forschungsprojekt „Innovations- und Dialogmark...
20611,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Hauptseminar Innovations- und Dialogmarketing
21177,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Seminar Innovations- und Dialogmarketing
21420,Lehrstuhl für Marketing & Innovation,Veranstaltung von,Übung Dialogmarketing


In [13]:
%%time
get_answer('Was gibt es für Seminare von Daniel Baier?')

100%|████████████████████████████████████████████████████████████████████████████| 8800/8800 [00:01<00:00, 4605.13it/s]


['Daniel Baier', 'Seminar'] []
Wall time: 12.4 s


,source,relation,target
4913,Doktoranden- und Habilitandenseminar Innovatio...,Veranstaltungsart,Seminar
5230,Forschungsprojekt Data Mining im Marketing mit R,Veranstaltungsart,Seminar
5238,Forschungsprojekt Innovations- und Dialogmarke...,Veranstaltungsart,Seminar


In [14]:
%%time
get_answer('Was gibt es für Vorlesungen von Daniel Baier?')

100%|████████████████████████████████████████████████████████████████████████████| 8800/8800 [00:01<00:00, 4543.39it/s]

['Daniel Baier', 'Vorlesung'] []
Wall time: 2.41 s


,source,relation,target
5343,Grundlagen Marketing- und Dienstleistungsmanag...,Veranstaltungsart,Vorlesung
6524,Vorlesung Dialogmarketing,Veranstaltungsart,Vorlesung


In [15]:
%%time
get_answer('Wer ist Mitarbeiter am Lehrstuhl von Daniel Baier?')

['Daniel Baier'] ['Wissenschaftlicher Mitarbeiter']
Wall time: 49 ms


,source,relation,target
29659,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Benedikt Brand
29664,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Karolina Ewers
29669,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Cristopher Kopplin
29674,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Theresa Rausch


In [16]:
%%time
get_answer("Wer ist wissenschaftlicher Mitarbeiter am Lehrstuhl für Marketing und Innovation?")

100%|██████████████████████████████████████████████████████████████████████████| 36100/36100 [00:07<00:00, 4564.80it/s]


['Lehrstuhl für Marketing & Innovation'] ['Wissenschaftlicher Mitarbeiter']
Wall time: 53.7 s


,source,relation,target
29659,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Benedikt Brand
29664,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Karolina Ewers
29669,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Cristopher Kopplin
29674,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Theresa Rausch


In [17]:
%%time
get_answer("Wer ist wissenschaftlicher Mitarbeiter von Data Mining?")

100%|████████████████████████████████████████████████████████████████████████████| 3619/3619 [00:00<00:00, 4053.94it/s]


['Forschungsprojekt Data Mining im Marketing mit R'] ['Wissenschaftlicher Mitarbeiter']
Wall time: 31.5 s


,source,relation,target
29659,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Benedikt Brand
29664,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Karolina Ewers
29669,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Cristopher Kopplin
29674,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Theresa Rausch


In [18]:
%%time
get_answer('Wer ist wissenschaftlicher Mitarbeiter bei Daniel Baier?')

['Daniel Baier'] ['Wissenschaftlicher Mitarbeiter']
Wall time: 31 ms


,source,relation,target
29659,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Benedikt Brand
29664,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Karolina Ewers
29669,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Cristopher Kopplin
29674,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Theresa Rausch


In [19]:
%%time
get_answer('Wer ist wissenschaftlicher Mitarbaitär bei Daniel Baier?')

['Daniel Baier'] ['Wissenschaftlicher Mitarbeiter']
Wall time: 1.73 s


,source,relation,target
29659,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Benedikt Brand
29664,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Karolina Ewers
29669,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Cristopher Kopplin
29674,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Theresa Rausch


In [20]:
%%time
get_answer('An welchem Lehrstuhl arbeitet Daniel Baier?')

['Daniel Baier'] ['Lehrstuhlinhaber']
Wall time: 478 ms


,source,relation,target
14467,Daniel Baier,Lehrstuhlinhaber,Lehrstuhl für Marketing & Innovation


In [21]:
%%time
get_answer('Wer ist Daniel Baier?')

100%|████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 4268.06it/s]


['Daniel Baier'] []
Wall time: 30.7 s


,source,relation,target
14467,Daniel Baier,Lehrstuhlinhaber,Lehrstuhl für Marketing & Innovation
14468,Daniel Baier,E-Mail,daniel.baier@uni-bayreuth.de
14469,Daniel Baier,Telefon,49 (0)921 / 55-4340
14470,Daniel Baier,Fax,49 (0)921 / 55-4342
14471,Daniel Baier,Raum,B9 Raum: 19
19847,Daniel Baier,Dozent,Abschlussarbeitenkolloquium Innovations- und D...
20091,Daniel Baier,Dozent,Doktoranden- und Habilitandenseminar Innovatio...
20408,Daniel Baier,Dozent,Forschungsprojekt Data Mining im Marketing mit R
20416,Daniel Baier,Dozent,Forschungsprojekt Innovations- und Dialogmarke...
20521,Daniel Baier,Dozent,Grundlagen Marketing- und Dienstleistungsmanag...


In [22]:
%%time
get_answer('Welche research scientist gibt es am Lehrstuhl für Marketing & Innovation?')

100%|██████████████████████████████████████████████████████████████████████████| 64471/64471 [00:16<00:00, 3964.76it/s]

['Lehrstuhl für Marketing & Innovation'] ['Wissenschaftlicher Mitarbeiter']
Wall time: 16.3 s


,source,relation,target
29659,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Benedikt Brand
29664,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Karolina Ewers
29669,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Cristopher Kopplin
29674,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Theresa Rausch


In [23]:
%%time
get_answer('Wie lautet die E-Mail von Daniel Baier?')

['Daniel Baier'] ['E-Mail']
Wall time: 21 ms


,source,relation,target
14468,Daniel Baier,E-Mail,daniel.baier@uni-bayreuth.de


In [24]:
%%time
get_answer('Wo ist das Büro von Daniel Baier?')

['Daniel Baier'] ['Raum']
Wall time: 21 ms


,source,relation,target
14471,Daniel Baier,Raum,B9 Raum: 19


In [25]:
%%time
get_answer('Wie lautet die Telefonnummer von Daniel Baier?')

['Daniel Baier'] ['Telefon']
Wall time: 30 ms


,source,relation,target
14469,Daniel Baier,Telefon,49 (0)921 / 55-4340


In [26]:
%%time
get_answer('Wer ist Professor am Chair für Marketing und Innovation?')

100%|██████████████████████████████████████████████████████████████████████████| 18729/18729 [00:04<00:00, 4407.85it/s]


['Lehrstuhl für Marketing & Innovation'] ['Lehrstuhlinhaber']
Wall time: 32.6 s


,source,relation,target
29643,Lehrstuhl für Marketing & Innovation,Lehrstuhlinhaber,Daniel Baier


In [27]:
%%time
get_answer('Wer sind Mitarbeiter am Lehrstuhl von Nils Urbach?')

['Nils Urbach'] ['Wissenschaftlicher Mitarbeiter']
Wall time: 41 ms


,source,relation,target
27267,Wirtschaftsinformatik und Strategisches IT-Man...,Wissenschaftlicher Mitarbeiter,Julia Amend
27272,Wirtschaftsinformatik und Strategisches IT-Man...,Wissenschaftlicher Mitarbeiter,Laurin Arnold
27277,Wirtschaftsinformatik und Strategisches IT-Man...,Wissenschaftlicher Mitarbeiter,Tobias Guggenberger
27282,Wirtschaftsinformatik und Strategisches IT-Man...,Wissenschaftlicher Mitarbeiter,Jan Jöhnk
27287,Wirtschaftsinformatik und Strategisches IT-Man...,Wissenschaftlicher Mitarbeiter,Luis Lämmermann
27291,Wirtschaftsinformatik und Strategisches IT-Man...,Wissenschaftlicher Mitarbeiter,Peter Hofmann
27296,Wirtschaftsinformatik und Strategisches IT-Man...,Wissenschaftlicher Mitarbeiter,Benjamin Schellinger
27299,Wirtschaftsinformatik und Strategisches IT-Man...,Wissenschaftlicher Mitarbeiter,Vincent Schlatt
27303,Wirtschaftsinformatik und Strategisches IT-Man...,Wissenschaftlicher Mitarbeiter,Fabiane Völter


In [28]:
%%time
get_answer('An welchem Lehrstuhl arbeitet Nils Urbach?')

['Nils Urbach'] ['Lehrstuhlinhaber']
Wall time: 417 ms


,source,relation,target
12078,Nils Urbach,Lehrstuhlinhaber,Wirtschaftsinformatik und Strategisches IT-Man...


In [29]:
%%time
get_answer('Wie lautet die E-Mail vom Dozent von Forschungsprojekt Data Mining im Marketing mit R?')

100%|████████████████████████████████████████████████████████████████████████| 266644/266644 [01:07<00:00, 3975.44it/s]


['Forschungsprojekt Data Mining im Marketing mit R'] ['Dozent', 'E-Mail']
Wall time: 1min 43s


,source,relation,target
14468,Daniel Baier,E-Mail,daniel.baier@uni-bayreuth.de


In [30]:
%%time
get_answer('Wer ist wissenschaftlicher Mitarbeiter vom Dozent von Data Mining mit R?')

100%|██████████████████████████████████████████████████████████████████████████| 64471/64471 [00:13<00:00, 4812.43it/s]


['Forschungsprojekt Data Mining im Marketing mit R'] ['Wissenschaftlicher Mitarbeiter', 'Dozent']
Wall time: 44.8 s


,source,relation,target
29659,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Benedikt Brand
29664,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Karolina Ewers
29669,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Cristopher Kopplin
29674,Lehrstuhl für Marketing & Innovation,Wissenschaftlicher Mitarbeiter,Theresa Rausch


In [31]:
%%time
get_answer('Was gibt es am Montag zu essen?')

['Montag'] ['Mensa']
Wall time: 25 ms


,source,relation,target
14927,Montag,Mensa,Cremiges Kohlrabi-Paprika-Currygulasch an gedr...


In [32]:
%%time
get_answer('Wann ist die Prüfung in Dialogmarketing?')

['Dialogmarketing'] ['Klausur']
Wall time: 21 ms


,source,relation,target
14958,Dialogmarketing,Klausur,"Donnerstag, 15.10.20 11:30 – 12:30 Audimax"
